<a href="https://colab.research.google.com/github/tomonari-masada/course-nlp2020/blob/master/07_document_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 文書分類
* 今回は、fastTextのような学習済みの単語埋め込みは使わない。
* 単語埋め込み自体の学習も、ネットワークの重みの学習と同時におこなう。
* IMDbデータの準備も、PyTorchのtorchtextモジュールを使っておこなう。
* ネットワークへの入力は、単語埋め込みを、単語の出現順どおりに並べた列にする。
* そして、前向き計算のなかではじめて、単語埋め込みの平均をとることにする。
* 参考資料
 * https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb

## 07-00 下準備
* Google ColabのランタイムのタイプをGPUに変更しておこう。
 * 上のメニューの「ランタイム」→「ランタイムのタイプを変更」→「ハードウェア　アクセラレータ」から「GPU」を選択

## 07-01 torchtextを使ってIMDbデータを読み込む
* https://torchtext.readthedocs.io/en/latest/datasets.html

### 実験の再現性確保のための設定など
* torch.backends.cudnn.deterministicをTrueにするのは、こうしないと、GPU上での計算が毎回同じ値を与えないため。

In [1]:
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets

SEED = 123

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### torchtextのフィールド
* TEXTフィールドと、LABELフィールドという２種類のFieldオブジェクトのインスタンスを作る。
* TEXTフィールドは、テキストの前処理の仕方を決めておくのに使う。
 * batch_firstをTrueに設定するのが今回のポイント。
 * tokenizerは、デフォルトでは単にstring型のsplitメソッドを適用するだけになる。これは高速だが、tokenizationとしては雑。
* LABELフィールドは、ラベルの前処理に使う。

In [2]:
TEXT = data.Field(tokenize="spacy", batch_first=True)
LABEL = data.LabelField()

### IMDbデータセットを前処理しつつ読み込む
* TEXTフィールドでspaCyのtokenizationを使うように設定したので、少し時間がかかる。

In [3]:
train_dev_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

### 最初の文書を見てみる
* `vars`関数は、モジュール、クラス、インスタンス、あるいはそれ以外の`__dict__`属性を持つオブジェクトの、`__dict__`属性を辞書として返す組み込み関数。

In [4]:
print(vars(train_dev_data.examples[0]))

{'text': ['I', 'really', 'liked', 'this', 'movie', ',', 'and', 'went', 'back', 'to', 'see', 'it', 'two', 'times', 'more', 'within', 'a', 'week.<br', '/><br', '/>Ms', '.', 'Detmers', 'nailed', 'the', 'performance', '-', 'she', 'was', 'like', 'a', 'hungry', 'cat', 'on', 'the', 'prowl', ',', 'toying', 'with', 'her', 'prey', '.', 'She', 'lashes', 'out', 'in', 'rage', 'and', 'lust', ',', 'taking', 'a', '"', 'too', 'young', '"', 'lover', ',', 'and', 'crashing', 'hundreds', 'of', 'her', 'terrorist', 'fiancé', "'s", 'mother', "'s", 'pieces', 'of', 'fine', 'china', 'to', 'the', 'floor', '.', '<', 'br', '/><br', '/>The', 'film', 'was', 'full', 'of', 'beautiful', 'touches', '.', 'The', 'Maserati', ',', 'the', 'wonderful', 'wardrobe', ',', 'the', 'flower', 'boxes', 'along', 'the', 'rooftops', '.', 'I', 'particularly', 'enjoyed', 'the', 'ancient', 'Greek', 'class', 'and', 'the', 'recitation', 'of', "'", "Antigone'.<br", '/><br', '/>It', 'had', 'a', 'feeling', 'of', "'", 'Story', 'of', 'O', "'", '-'

In [5]:
print(train_dev_data.examples[0].text)

['I', 'really', 'liked', 'this', 'movie', ',', 'and', 'went', 'back', 'to', 'see', 'it', 'two', 'times', 'more', 'within', 'a', 'week.<br', '/><br', '/>Ms', '.', 'Detmers', 'nailed', 'the', 'performance', '-', 'she', 'was', 'like', 'a', 'hungry', 'cat', 'on', 'the', 'prowl', ',', 'toying', 'with', 'her', 'prey', '.', 'She', 'lashes', 'out', 'in', 'rage', 'and', 'lust', ',', 'taking', 'a', '"', 'too', 'young', '"', 'lover', ',', 'and', 'crashing', 'hundreds', 'of', 'her', 'terrorist', 'fiancé', "'s", 'mother', "'s", 'pieces', 'of', 'fine', 'china', 'to', 'the', 'floor', '.', '<', 'br', '/><br', '/>The', 'film', 'was', 'full', 'of', 'beautiful', 'touches', '.', 'The', 'Maserati', ',', 'the', 'wonderful', 'wardrobe', ',', 'the', 'flower', 'boxes', 'along', 'the', 'rooftops', '.', 'I', 'particularly', 'enjoyed', 'the', 'ancient', 'Greek', 'class', 'and', 'the', 'recitation', 'of', "'", "Antigone'.<br", '/><br', '/>It', 'had', 'a', 'feeling', 'of', "'", 'Story', 'of', 'O', "'", '-', 'that',

In [6]:
print(train_dev_data.examples[0].label)

pos


### テストセット以外の部分を訓練データと検証データに分ける

In [7]:
train_data, dev_data = train_dev_data.split(split_ratio=0.8, random_state = random.seed(SEED))

In [8]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of development examples: {len(dev_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 20000
Number of development examples: 5000
Number of testing examples: 25000


### データセットのラベルを作る
* TEXTラベルのほうでは、最大語彙サイズを指定する。

In [9]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [10]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


### 出現頻度順で上位２０単語を見てみる

In [11]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 232028), (',', 219642), ('.', 189279), ('and', 125246), ('a', 124915), ('of', 115025), ('to', 106919), ('is', 87460), ('in', 70029), ('I', 62000), ('it', 61196), ('that', 56281), ('"', 50475), ("'s", 49438), ('this', 48574), ('-', 42352), ('/><br', 40924), ('was', 39912), ('as', 34619), ('with', 34266)]


### 単語ID順に最初の１０単語を見てみる
* IDのうち、0と1は、未知語とパディング用の単語という特殊な単語に割り振られている。
 * パディングとは、長さが不揃いの複数の文書を同じミニバッチにまとめるとき、すべての文書の長さを無理やりそろえるため、文書末尾に特殊な単語（元々の語彙にない、人工的に用意した単語）を追加すること。

In [12]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


### ラベルのほうのIDを確認する
* こちらはnegとposに対応する２つのIDしかない。

In [13]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f35154642f0>, {'neg': 0, 'pos': 1})


### ミニバッチを取り出すためのiteratorを作る
* ミニバッチのサイズを指定する。
 * ミニバッチのサイズは、性能を出すためにチューニングすべきハイパーパラメータのひとつ。

In [14]:
BATCH_SIZE = 100

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, dev_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, dev_data, test_data),
    batch_size=BATCH_SIZE,
    device=device)

### 試しにテストセットのiteratorを回してミニバッチをすべて取得して個数を数えてみる

In [15]:
i = 0
for batch in test_iterator:
  i += 1
  continue
print(f'We have {i} mini-batches in test set.')

We have 250 mini-batches in test set.


### ミニバッチの中身を見てみる

上記のループを抜けたあとには、変数batchにはテストセットの最後のミニバッチが代入されている。

そこでこの最後のミニバッチのshapeを確認する。

In [16]:
batch.text.shape

torch.Size([100, 2640])

このミニバッチに含まれる文書のうち、最初のものの単語ID列と、先頭100個のIDを単語に戻したものを表示させてみる。

In [17]:
print(batch.text[0])
print(' '.join([TEXT.vocab.itos[i] for i in batch.text[0][:100]]))

tensor([152,  15,   6,  ...,   7, 324,   4], device='cuda:0')
There 's a sign on The Lost Highway that <unk> /><br <unk> SPOILERS <unk> /><br <unk> you already knew that , did n't <unk> /><br />Since there 's a great deal of people that apparently did not get the point of this movie , I 'd like to contribute my interpretation of why the plot makes perfect sense . As others have pointed out , one single viewing of this movie is not sufficient . If you have the DVD of <unk> , you can " cheat " by looking at David Lynch 's " Top 10 <unk> to <unk>


最後の文書の末尾は「1」で埋められていることが分かる。

In [18]:
print(batch.text[BATCH_SIZE-1])

tensor([  14,   25, 2884,  ...,    1,    1,    1], device='cuda:0')


ミニバッチに含まれる文書の長さを調べると、文書が文書長の降順に並べられていることが分かる。

In [19]:
(batch.text != 1).sum(1)

tensor([2640, 2537, 2534, 1915, 1635, 1364, 1324, 1290, 1287, 1284, 1279, 1274,
        1264, 1255, 1253, 1241, 1241, 1232, 1229, 1226, 1225, 1221, 1213, 1212,
        1211, 1210, 1206, 1205, 1198, 1196, 1195, 1193, 1193, 1193, 1191, 1190,
        1190, 1187, 1185, 1183, 1182, 1181, 1181, 1181, 1181, 1179, 1179, 1177,
        1176, 1175, 1175, 1175, 1173, 1173, 1173, 1172, 1172, 1172, 1171, 1170,
        1169, 1169, 1168, 1167, 1166, 1166, 1164, 1164, 1163, 1163, 1162, 1161,
        1159, 1158, 1158, 1157, 1157, 1155, 1155, 1153, 1152, 1152, 1151, 1150,
        1149, 1148, 1147, 1146, 1145, 1144, 1144, 1144, 1144, 1144, 1144, 1143,
        1143, 1142, 1142, 1142], device='cuda:0')

## 07-02 MLPによる文書分類の準備

### 定数の設定

In [20]:
INPUT_DIM = len(TEXT.vocab)
NUM_CLASS = len(LABEL.vocab)
EMBED_DIM = 100
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

### モデルを定義する前にPyTorchの単語埋め込みがどんなものかを見てみる

以下のように、語彙サイズと埋め込み次元数を指定しつつ、torch.nn.Embeddingのインスタンスを作ればよい。

In [21]:
embed = nn.Embedding(INPUT_DIM, EMBED_DIM, padding_idx=PAD_IDX)

パディング用の単語の埋め込みはゼロベクトルになる。

In [22]:
print(embed(torch.tensor([[0,2,1],[2,3,4]])))

tensor([[[ 0.3374, -0.1778, -0.3035, -0.5880,  0.3486,  0.6603, -0.2196,
          -0.3792,  0.7671, -1.1925,  0.6984, -1.4097,  0.1794,  1.8951,
           0.4954,  0.2692, -0.0770, -1.0205, -0.1690,  0.9178,  1.5810,
           1.3010,  1.2753, -0.2010,  0.4965, -1.5723,  0.9666, -1.1481,
          -1.1589,  0.3255, -0.6315, -2.8400, -1.3250,  0.1784, -2.1338,
           1.0524, -0.3885, -0.9343, -0.4991, -1.0867,  0.8805,  1.5542,
           0.6266, -0.1755,  0.0983, -0.0935,  0.2662, -0.5850,  0.8768,
           1.6221, -1.4779,  1.1331, -1.2203,  1.3139,  1.0533,  0.1388,
           2.2473, -0.8036, -0.2808,  0.7697, -0.6596, -0.7979,  0.1838,
           0.2293,  0.5146,  0.9938, -0.2587, -1.0826, -0.0444,  1.6236,
          -2.3229,  1.0878,  0.6716,  0.6933, -0.9487, -0.0765, -0.1526,
           0.1167,  0.4403, -1.4465,  0.2553, -0.5496,  1.0042,  0.8272,
          -0.3948,  0.4892, -0.2168, -1.7472, -1.6025, -1.0764,  0.9031,
          -0.7218, -0.5951, -0.7112,  0.6230, -1.37

### モデルの定義
* 基本的にMLPだが、入り口に単語埋め込み層が挿入されている。

In [23]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx):
    super(EmbedTextSentiment, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)
    self.init_weights()

  def init_weights(self):
    initrange = 0.5
    self.fc1.weight.data.uniform_(-initrange, initrange)
    self.fc1.bias.data.zero_()
    self.fc2.weight.data.uniform_(-initrange, initrange)
    self.fc2.bias.data.zero_()
    self.fc3.weight.data.uniform_(-initrange, initrange)
    self.fc3.bias.data.zero_()

  def forward(self, text):
    x = self.embed(text)
    x = x.mean(1) 
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

### モデルを作る
* モデル（のインスタンス）をGPUに移動させている点に注意。

In [24]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX).to(device)

### 損失関数とoptimizerとschedulerを作る
* 損失関数をGPUに移動させている点に注意。

In [25]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

### 訓練用の関数

In [26]:
def train(data_iterator, model, optimizer, scheduler, criterion):

  model.train()

  train_loss = 0
  train_acc = 0
  for batch in data_iterator:
    optimizer.zero_grad()
    text, cls = batch.text, batch.label
    output = model(text)
    loss = criterion(output, cls)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    train_acc += (output.argmax(1) == cls).sum().item()

  scheduler.step()

  data_len = len(data_iterator.dataset)
  return train_loss / data_len, train_acc / data_len

### 評価用の関数

In [27]:
def test(data_iterator, model, criterion):

  model.eval()

  loss = 0
  acc = 0
  for batch in data_iterator:
    text, cls = batch.text, batch.label
    with torch.no_grad():
      output = model(text)
      loss = criterion(output, cls)
      loss += loss.item()
      acc += (output.argmax(1) == cls).sum().item()

  data_len = len(data_iterator.dataset)
  return loss / data_len, acc / data_len

## 07-03 分類器の訓練と開発セットでの評価

In [28]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  dev_loss, dev_acc = test(dev_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60

  print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {dev_loss:.4f}(dev)\t|\tAcc: {dev_acc * 100:.1f}%(dev)')

Epoch: 1  | time in 0 minutes, 4 seconds
	Loss: 0.0043(train)	|	Acc: 79.9%(train)
	Loss: 0.0001(dev)	|	Acc: 88.5%(dev)
Epoch: 2  | time in 0 minutes, 4 seconds
	Loss: 0.0016(train)	|	Acc: 93.9%(train)
	Loss: 0.0001(dev)	|	Acc: 87.4%(dev)
Epoch: 3  | time in 0 minutes, 4 seconds
	Loss: 0.0008(train)	|	Acc: 97.4%(train)
	Loss: 0.0002(dev)	|	Acc: 87.0%(dev)
Epoch: 4  | time in 0 minutes, 4 seconds
	Loss: 0.0004(train)	|	Acc: 98.6%(train)
	Loss: 0.0003(dev)	|	Acc: 86.7%(dev)
Epoch: 5  | time in 0 minutes, 4 seconds
	Loss: 0.0002(train)	|	Acc: 99.4%(train)
	Loss: 0.0003(dev)	|	Acc: 86.8%(dev)
Epoch: 6  | time in 0 minutes, 4 seconds
	Loss: 0.0001(train)	|	Acc: 99.7%(train)
	Loss: 0.0004(dev)	|	Acc: 87.1%(dev)
Epoch: 7  | time in 0 minutes, 4 seconds
	Loss: 0.0001(train)	|	Acc: 99.8%(train)
	Loss: 0.0004(dev)	|	Acc: 86.9%(dev)
Epoch: 8  | time in 0 minutes, 4 seconds
	Loss: 0.0000(train)	|	Acc: 99.9%(train)
	Loss: 0.0005(dev)	|	Acc: 87.0%(dev)
Epoch: 9  | time in 0 minutes, 4 seconds
	Loss: 

## 07-04 再検討
* 訓練データ上での分類精度が100%になってしまっている。
 * 明らかにオーバーフィッティングを起こしてしまっている。

### ドロップアウトを使う
* モデルのインスタンスを作るときにdropoutの確率を引数pで指定できるようにする。

In [29]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx, p=0.0):
    super(EmbedTextSentiment, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.dropout = nn.Dropout(p=p)
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)
    self.init_weights()

  def init_weights(self):
    initrange = 0.5
    self.fc1.weight.data.uniform_(-initrange, initrange)
    self.fc1.bias.data.zero_()
    self.fc2.weight.data.uniform_(-initrange, initrange)
    self.fc2.bias.data.zero_()
    self.fc3.weight.data.uniform_(-initrange, initrange)
    self.fc3.bias.data.zero_()

  def forward(self, text):
    x = self.dropout(self.embed(text))
    x = x.mean(1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [30]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

In [31]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  dev_loss, dev_acc = test(dev_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60

  print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {dev_loss:.4f}(dev)\t|\tAcc: {dev_acc * 100:.1f}%(dev)')

Epoch: 1  | time in 0 minutes, 4 seconds
	Loss: 0.0050(train)	|	Acc: 75.3%(train)
	Loss: 0.0001(dev)	|	Acc: 88.0%(dev)
Epoch: 2  | time in 0 minutes, 4 seconds
	Loss: 0.0024(train)	|	Acc: 90.2%(train)
	Loss: 0.0001(dev)	|	Acc: 88.9%(dev)
Epoch: 3  | time in 0 minutes, 4 seconds
	Loss: 0.0016(train)	|	Acc: 93.8%(train)
	Loss: 0.0001(dev)	|	Acc: 88.4%(dev)
Epoch: 4  | time in 0 minutes, 4 seconds
	Loss: 0.0012(train)	|	Acc: 95.4%(train)
	Loss: 0.0001(dev)	|	Acc: 88.9%(dev)
Epoch: 5  | time in 0 minutes, 4 seconds
	Loss: 0.0009(train)	|	Acc: 96.5%(train)
	Loss: 0.0001(dev)	|	Acc: 88.4%(dev)
Epoch: 6  | time in 0 minutes, 4 seconds
	Loss: 0.0007(train)	|	Acc: 97.4%(train)
	Loss: 0.0002(dev)	|	Acc: 88.6%(dev)
Epoch: 7  | time in 0 minutes, 4 seconds
	Loss: 0.0006(train)	|	Acc: 97.5%(train)
	Loss: 0.0001(dev)	|	Acc: 88.6%(dev)
Epoch: 8  | time in 0 minutes, 4 seconds
	Loss: 0.0005(train)	|	Acc: 98.2%(train)
	Loss: 0.0002(dev)	|	Acc: 88.3%(dev)
Epoch: 9  | time in 0 minutes, 4 seconds
	Loss: 

### L２正則化を使う
* optimizerのweight_decayパラメータを0より大きな値にする。

In [32]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

In [33]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  dev_loss, dev_acc = test(dev_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60

  print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {dev_loss:.4f}(dev)\t|\tAcc: {dev_acc * 100:.1f}%(dev)')

Epoch: 1  | time in 0 minutes, 4 seconds
	Loss: 0.0066(train)	|	Acc: 62.8%(train)
	Loss: 0.0002(dev)	|	Acc: 75.9%(dev)
Epoch: 2  | time in 0 minutes, 4 seconds
	Loss: 0.0051(train)	|	Acc: 75.7%(train)
	Loss: 0.0001(dev)	|	Acc: 81.7%(dev)
Epoch: 3  | time in 0 minutes, 4 seconds
	Loss: 0.0045(train)	|	Acc: 79.3%(train)
	Loss: 0.0001(dev)	|	Acc: 84.3%(dev)
Epoch: 4  | time in 0 minutes, 4 seconds
	Loss: 0.0041(train)	|	Acc: 82.7%(train)
	Loss: 0.0002(dev)	|	Acc: 81.4%(dev)
Epoch: 5  | time in 0 minutes, 4 seconds
	Loss: 0.0039(train)	|	Acc: 83.6%(train)
	Loss: 0.0001(dev)	|	Acc: 85.6%(dev)
Epoch: 6  | time in 0 minutes, 4 seconds
	Loss: 0.0036(train)	|	Acc: 85.1%(train)
	Loss: 0.0001(dev)	|	Acc: 85.4%(dev)
Epoch: 7  | time in 0 minutes, 4 seconds
	Loss: 0.0036(train)	|	Acc: 85.3%(train)
	Loss: 0.0001(dev)	|	Acc: 86.9%(dev)
Epoch: 8  | time in 0 minutes, 4 seconds
	Loss: 0.0034(train)	|	Acc: 86.3%(train)
	Loss: 0.0002(dev)	|	Acc: 85.9%(dev)
Epoch: 9  | time in 0 minutes, 4 seconds
	Loss: 

### early stopping
* dev setでのaccuracyが4回連続で最高値を下回ったら訓練を終えることにする。
* early stoppingの実現については、PyTorch Lightningを使う手もある。
 * https://pytorch-lightning.readthedocs.io/en/latest/early_stopping.html

In [34]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

In [35]:
patience = 4
early_stop_count = 0
best_dev_acc = 0.0
dev_acc_threshold = 0.87

N_EPOCHS = 50 # エポック数を増やしておく
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  dev_loss, dev_acc = test(dev_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60

  print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {dev_loss:.4f}(dev)\t|\tAcc: {dev_acc * 100:.1f}%(dev)')

  # early stopping
  if best_dev_acc <= dev_acc:
    best_dev_acc = dev_acc
    early_stop_count = 0
  else:
    early_stop_count += 1
    if early_stop_count == patience:
      break

Epoch: 1  | time in 0 minutes, 4 seconds
	Loss: 0.0066(train)	|	Acc: 63.9%(train)
	Loss: 0.0002(dev)	|	Acc: 75.1%(dev)
Epoch: 2  | time in 0 minutes, 4 seconds
	Loss: 0.0051(train)	|	Acc: 75.3%(train)
	Loss: 0.0001(dev)	|	Acc: 80.8%(dev)
Epoch: 3  | time in 0 minutes, 4 seconds
	Loss: 0.0046(train)	|	Acc: 78.7%(train)
	Loss: 0.0002(dev)	|	Acc: 79.6%(dev)
Epoch: 4  | time in 0 minutes, 4 seconds
	Loss: 0.0041(train)	|	Acc: 81.7%(train)
	Loss: 0.0001(dev)	|	Acc: 82.8%(dev)
Epoch: 5  | time in 0 minutes, 4 seconds
	Loss: 0.0039(train)	|	Acc: 83.4%(train)
	Loss: 0.0001(dev)	|	Acc: 85.5%(dev)
Epoch: 6  | time in 0 minutes, 4 seconds
	Loss: 0.0037(train)	|	Acc: 84.3%(train)
	Loss: 0.0001(dev)	|	Acc: 85.7%(dev)
Epoch: 7  | time in 0 minutes, 4 seconds
	Loss: 0.0035(train)	|	Acc: 86.0%(train)
	Loss: 0.0001(dev)	|	Acc: 85.1%(dev)
Epoch: 8  | time in 0 minutes, 4 seconds
	Loss: 0.0034(train)	|	Acc: 85.9%(train)
	Loss: 0.0001(dev)	|	Acc: 86.5%(dev)
Epoch: 9  | time in 0 minutes, 4 seconds
	Loss: 

ここで戻ってweight_decayやdropoutの確率などをチューニングし直す。



---



## 07-05 テストセット上で評価
* 見つけ出したベストな設定を使って、テストセット上での最終的な評価をおこなう。

In [36]:
# ベストな設定を使っての学習のやり直しのコードをここに書く

In [37]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_iterator, model, criterion)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0000(test)	|	Acc: 88.3%(test)
